In [3]:
import numpy as np
import imageio.v3 as iio
import os
import HREBSD as hr
import utilities as ut
import matplotlib.pyplot as plt

# Load the data
folder = "D:/Research/CoNi67/Data/HREBSD/"
paths = [f for f in os.listdir(folder) if f.endswith('.tif')]
data = np.zeros((10, 10, 1024, 1024), dtype=np.uint16)
for path in paths:
    pat = iio.imread(folder + path)
    pos_info = path.replace(".tif", "").split("_")
    if "x" in pos_info[0]:
        x = int(pos_info[0].replace("x", ""))
        y = int(pos_info[1].replace("y", ""))
    else:
        x = int(pos_info[1].replace("x", ""))
        y = int(pos_info[0].replace("y", ""))
    data[y, x] = pat

PC = (0.492224, 0.546017, 0.971476)
delta = 6.5
N = (2048, 2048)
PC = ut.convert_pc(PC, N, delta)

In [2]:
ref = data[0, 0]
exp = data[0, 1]
out = hr.cross_correlation_function(ref, exp)
print(out)

(array([[2.37643233e+08-0.0293123j , 5.41245670e+08-0.01444273j,
        8.60907768e+08-0.04251719j, ..., 6.67959665e+08-0.02012568j,
        3.86237465e+08-0.01827766j, 1.62920010e+08-0.0453287j ],
       [3.98774950e+08-0.02944955j, 9.14727314e+08-0.01460103j,
        1.45852061e+09-0.04261828j, ..., 1.15953391e+09-0.02023599j,
        7.42224372e+08-0.018426j  , 2.95061254e+08-0.04544367j],
       [6.81086406e+08-0.02977512j, 1.41306286e+09-0.01486019j,
        2.22577093e+09-0.04284103j, ..., 1.79749331e+09-0.02047732j,
        1.12763391e+09-0.01868355j, 4.91366749e+08-0.04564322j],
       ...,
       [1.76852484e+09-0.02871762j, 3.78564811e+09-0.013933j  ,
        6.44393255e+09-0.04204196j, ..., 5.05832338e+09-0.01974168j,
        2.74839117e+09-0.01758149j, 1.47004405e+09-0.04495985j],
       [7.84700298e+08-0.02893935j, 1.65814392e+09-0.01403611j,
        3.05567082e+09-0.04229355j, ..., 2.97543590e+09-0.01982534j,
        1.37279924e+09-0.01782385j, 7.00962082e+08-0.04511159j